In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
data_addr = '/content/drive/MyDrive/cs5284pro/dow_1day_price.csv'
adj_addr = '/content/drive/MyDrive/cs5284pro/dow_1day_090_01_corr.csv'
s_index = 0
lr = 1e-3
n_neurons = 128
seq_len = 12
n_epochs = 100
batch_size = 128
n_off = 0
th = 0.2

In [28]:
import pandas as pd
import scipy
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, mean_absolute_error
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import scipy.sparse as sp
from torch.utils.data import TensorDataset, DataLoader


class GCGRUCell(nn.Module):
    def __init__(self, num_units, adj, num_gcn_nodes, s_index):
        super(GCGRUCell, self).__init__()
        self.units = num_units
        self._gcn_nodes = num_gcn_nodes
        self.s_index = s_index

        # Preprocess adjacency matrix
        adj = self.calculate_laplacian(adj)
        # if isinstance(adj, sp.coo_matrix):
        # adj = adj.toarray()  # Convert to a dense NumPy array
        self.register_buffer("_adj", adj)  # Save as a non-trainable buffer

        # GCN weights
        self.w0 = nn.Parameter(torch.randn(1, self.units))

        # GRU weights
        self.wz = nn.Linear(self.units, self.units, bias=True)
        self.wr = nn.Linear(self.units, self.units, bias=True)
        self.wh = nn.Linear(self.units, self.units, bias=True)

        self.uz = nn.Linear(self.units, self.units, bias=False)
        self.ur = nn.Linear(self.units, self.units, bias=False)
        self.uh = nn.Linear(self.units, self.units, bias=False)

    @property
    def state_size(self):
        return self.units

    def calculate_laplacian(self, adj):
        adj = adj + torch.eye(adj.size(0))  # Add self-loops
        degree = torch.sum(adj, dim=1)
        degree[degree==0] = 1
        d_inv_sqrt = torch.diag(torch.pow(degree, -0.5))
        laplacian = d_inv_sqrt @ adj @ d_inv_sqrt
        return laplacian

    def gc(self, inputs):
        # Graph convolution: Ax
        ax = torch.matmul(inputs, self._adj)

        # Select specific nodes
        ax = ax[:, self.s_index]

        # Expand last dimension
        ax = ax.unsqueeze(-1)

        # Transform features
        return torch.matmul(ax, self.w0)

    def forward(self, inputs, state):
        x = self.gc(inputs)

        # GRU gates
        z = torch.sigmoid(self.wz(x) + self.uz(state))
        r = torch.sigmoid(self.wr(x) + self.ur(state))
        h = torch.tanh(self.wh(x) + self.uh(r * state))

        # Update state
        output = z * state + (1 - z) * h
        return output



In [29]:
class GCGRUModel(nn.Module):
    def __init__(self, cell, seq_len, num_gcn_nodes):
        super(GCGRUModel, self).__init__()
        self.cell = cell
        self.seq_len = seq_len
        self.num_gcn_nodes = num_gcn_nodes

    def forward(self, x):
        batch_size = x.size(0)
        state = torch.zeros(batch_size, self.cell.units).to(x.device)

        outputs = []
        for t in range(self.seq_len):
            state = self.cell(x[:, t, :], state)
            outputs.append(state)

        return torch.stack(outputs, dim=1)  # (batch_size, seq_len, units)


In [30]:
def normalize_adj(adj):
    """Normalized adjacency matrix A_hat = D^-0.5 A D^-0.5"""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    normalized_adj = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()
    return normalized_adj.astype(np.float32)

def load_dow_price_data(data_addr, adj_addr):
    data = pd.read_csv(data_addr).values
    adj = pd.read_csv(adj_addr, header=None).values
    # data = normalize(data, axis=0)
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)
    return data, adj

def preprocess_data(data, labels, time_len, train_rate, seq_len, pre_len):
    X, Y, pre_Y = [], [], []
    for i in range(time_len - seq_len - pre_len):
        X.append(data[i:i + seq_len, :])
        Y.append(labels[i + seq_len:i + seq_len + pre_len])
        pre_Y.append(labels[(i + seq_len - 1):(i + seq_len + pre_len - 1)])

    # Split the training set and test set
    train_size = int(train_rate * len(X))
    X_train = np.array(X[:train_size])
    Y_train = np.array(Y[:train_size])
    X_test = np.array(X[train_size:])
    Y_test = np.array(Y[train_size:])
    # pre_Y_test = labels[train_size + seq_len:train_size + seq_len + len(X_test)]
    pre_Y_test = np.array(pre_Y[train_size:])

    return X_train, Y_train, X_test, Y_test, pre_Y_test

In [31]:

# Load and preprocess data
data, adj = load_dow_price_data(data_addr, adj_addr)
adj = normalize_adj(adj)
if isinstance(adj, sp.coo_matrix):
    adj = adj.toarray()  # Convert to a dense NumPy array
labels = data[:, s_index]
if n_off > 0:
    data = data[:-n_off]
    labels = labels[n_off:]

train_rate = 0.8
pre_len = 1
time_len = data.shape[0]
n_gcn_nodes = data.shape[1]

X_train, y_train, X_test, y_test, pre_y_test = preprocess_data( data, labels, time_len, train_rate, seq_len, pre_len)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)




<ipython-input-30-ae657d590bf0>:5: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


In [32]:
# Initialize model
cell = GCGRUCell(n_neurons, torch.tensor(adj, dtype=torch.float32), n_gcn_nodes, s_index)
model = GCGRUModel(cell, seq_len, n_gcn_nodes)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()
# Training
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train).squeeze()
    loss = criterion(outputs[:, -1, 0].unsqueeze(1), y_train) # Select the first output node for comparison with y_train
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {loss.item()}")


Epoch 1/100, Loss: 0.3466346263885498
Epoch 2/100, Loss: 0.12062768638134003
Epoch 3/100, Loss: 0.019313275814056396
Epoch 4/100, Loss: 0.004977640695869923
Epoch 5/100, Loss: 0.02359420619904995
Epoch 6/100, Loss: 0.04325191304087639
Epoch 7/100, Loss: 0.054792653769254684
Epoch 8/100, Loss: 0.058269768953323364
Epoch 9/100, Loss: 0.055615611374378204
Epoch 10/100, Loss: 0.04879026859998703
Epoch 11/100, Loss: 0.03952440619468689
Epoch 12/100, Loss: 0.029365815222263336
Epoch 13/100, Loss: 0.019721608608961105
Epoch 14/100, Loss: 0.01182551309466362
Epoch 15/100, Loss: 0.006611766759306192
Epoch 16/100, Loss: 0.00449924822896719
Epoch 17/100, Loss: 0.005169584881514311
Epoch 18/100, Loss: 0.00760465394705534
Epoch 19/100, Loss: 0.010456779040396214
Epoch 20/100, Loss: 0.012524611316621304
Epoch 21/100, Loss: 0.013134594075381756
Epoch 22/100, Loss: 0.012241979129612446
Epoch 23/100, Loss: 0.010275186970829964
Epoch 24/100, Loss: 0.0078697195276618
Epoch 25/100, Loss: 0.005622196476906

In [33]:
# Evaluation
model.eval()
with torch.no_grad():
    predictions = model(X_test)

    result = predictions[:, -1, 0].unsqueeze(1).numpy()

# Metrics
# print(result.shape)
# print(y_test.shape)
r2 = r2_score(y_test, result)
rmse = sqrt(mean_squared_error(y_test, result))
mae = mean_absolute_error(y_test, result)
# re = avg_relative_error(y_test, result)

print("***********************")
print(f"R2: {r2}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
# print(f"Relative Error: {re}")

***********************
R2: 0.9845036268234253
RMSE: 0.023273538984219584
MAE: 0.017209293320775032
